In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import minmax_scale

In [2]:
# #PATH
path_info = "F:/178/Tugas Akhir/dataset/annotations/"
path_to_fitur = "F:/178/Tugas Akhir/dataset/coba fitur/"
path_to_experiment ="F:/178/Tugas Akhir/dataset/experiment/"
path_to_model = "F:/178/Tugas Akhir/dataset/model/"
path_to_predict = "F:/178/Tugas Akhir/dataset/predict/"

# path_info = "C:/Users/USER/Downloads/Tugas Akhir/dataset/annotations/"
# path_to_fitur = "C:/Users/USER/Downloads/Tugas Akhir/dataset/coba fitur/"
# path_to_experiment ="C:/Users/USER/Downloads/Tugas Akhir/dataset/experiment/"
# path_to_model = "C:/Users/USER/Downloads/Tugas Akhir/dataset/model/"
# path_to_predict = "C:/Users/USER/Downloads/Tugas Akhir/dataset/predict/"

In [3]:
def load(fitur):
    data = pd.read_csv(path_to_experiment + fitur+".csv",header=None)
    ground = pd.read_csv(path_info + "static_annotations.csv")

    arousal_mean = ground['mean_arousal']
    valence_mean = ground['mean_valence']
    
    return data,arousal_mean,valence_mean

def split(data,value):
    train_data, test_data, train_values, test_values = train_test_split(data,value,test_size=0.3,random_state=1000)
    
    return train_data, test_data, train_values, test_values
    
def predict(data,values,test,true,clf):
    trained = clf.fit(data,values)
    pred = clf.predict(test)
    df = pd.DataFrame(data=[pred,true])
    
    return pred,trained,df

def score(pred,true):
    r2 = r2_score(true, pred)
    
    return r2

def regresion(clf,data,values):
    scores = cross_val_score(clf,data,values,scoring='r2',cv=10)
    
    return scores

def save_model(model,case,types):
    path = path_to_model+types+case+'norm_svr.sav' # save the model
    joblib.dump(model, path)

def save_predict(df1,df2,case):
    path = path_to_predict+case+"_norm_svr.xlsx" # save the model
    arousal = df1.T
    valence = df2.T
    merged = arousal.merge(valence, how='outer', left_index=True, right_index=True)
    merged.to_excel(path,header=None,index=None)

def norm(data):
    scaler.fit(data)
    normalized = scaler.transform(data)
    
    return normalized

In [7]:
kernels = RationalQuadratic(length_scale=1.0, alpha=1.0, 
                            length_scale_bounds=(1e-3, 1e3), 
                            alpha_bounds=(1e-3, 1e3))

gpr = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=15, 
                    normalize_y=True, copy_X_train=False, 
                    random_state=1000) #init Arousal

# CASE 1

In [8]:
datafitur = "case1" #CASE 1

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

#split arousal
train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1)))
#split valence
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) 

(744, 26)


INITIAL MODEL

In [6]:
# gpr = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Arousal

# svr_case1_v = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Valence

In [9]:
print("arousal:\t",gpr.kernel)
# print("valence:\t",svr_case1.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [10]:
# #Predict Arousal
a_a,train_a,df1_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, gpr)
# #Predict Valence
a_v,train_v,df1_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, gpr)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

In [11]:
print("arousal:\t",train_a.kernel_)
print("valence:\t",train_v.kernel_)

arousal:	 RationalQuadratic(alpha=0.0311, length_scale=2)
valence:	 RationalQuadratic(alpha=0.0311, length_scale=2)


AROUSAL

In [12]:
df1_a = df1_a.round(2)

display(df1_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.09,0.24,0.58,-0.41,-0.41,-0.09,-0.81,0.26,-0.44,-0.09,...,0.49,0.12,0.00,-0.16,-0.06,-0.00,0.32,0.36,-0.23,-0.45
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.64


VALENCE

In [13]:
df1_v = df1_v.round(2)

display(df1_v.head())
print("R2 Score Valence: %0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.18,0.32,0.49,-0.15,-0.07,0.11,-0.32,0.23,-0.07,0.04,...,0.43,0.33,0.01,-0.04,-0.01,0.05,0.16,-0.00,-0.14,-0.45
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence: 0.29


REGRESSION

In [12]:
# scores_a = regresion(svr_case1,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
# scores_v = regresion(svr_case1,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [13]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [14]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [15]:
# save_predict(df1_a.round(2),df1_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 2

In [14]:
datafitur = "case2" #CASE 2

data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 34)


INITIAL MODEL

In [15]:
# gpr = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Arousal

svr_case1_v = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=5, 
                    normalize_y=True, copy_X_train=True, 
                    random_state=1000) #init Valence

svr_case1_a = GPR(kernel=kernels, alpha=1e-10, 
                    optimizer='fmin_l_bfgs_b', 
                    n_restarts_optimizer=5, 
                    normalize_y=True, copy_X_train=True, 
                    random_state=1000) #init Valence

In [16]:
print("arousal:\t",svr_case1_a.kernel)
print("valence:\t",svr_case1_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)
valence:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [17]:
# #Predict Arousal
a_a,train_a,df2_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, svr_case1_a)
# #Predict Valence
a_v,train_v,df2_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, svr_case1_v)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

In [18]:
print("arousal:\t",train_a.kernel_)
print("valence:\t",train_v.kernel_)

arousal:	 RationalQuadratic(alpha=0.0296, length_scale=5.29)
valence:	 RationalQuadratic(alpha=0.0301, length_scale=3.23)


AROUSAL

In [19]:
df2_a = df2_a.round(2)

display(df2_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.02,0.17,0.47,-0.52,-0.38,-0.16,-0.84,0.27,-0.30,-0.08,...,0.47,0.11,-0.28,-0.29,-0.06,0.03,0.32,0.44,-0.23,-0.32
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.67


VALENCE

In [20]:
df2_v = df2_v.round(2)

display(df2_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.22,0.23,0.40,-0.40,-0.09,-0.01,-0.36,0.32,0.18,0.11,...,0.47,0.50,-0.28,-0.19,-0.05,0.08,0.17,0.12,-0.32,-0.14
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.37


REGRESSION

In [23]:
# scores_a = regresion(svr_case2,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
# scores_v = regresion(svr_case2,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [24]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [25]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [26]:
# save_predict(df2_a.round(2),df2_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 3

In [27]:
datafitur =  "case3" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 37)


INITAL MODEL

In [28]:
# svr_case3 = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Arousal

# svr_case1_v = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Valence

In [29]:
print("arousal:\t",gpr.kernel)
# print("valence:\t",svr_case1_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [30]:
# #Predict Arousal
a_a,train_a,df3_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, gpr)
# #Predict Valence
a_v,train_v,df3_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, gpr)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

C:\Users\5115100178\AppData\Local\Continuum\Anaconda3\envs\py36\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00130425, -0.30822961]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 73, 'nit': 5, 'warnflag': 2}
  ConvergenceWarning)


In [31]:
print("arousal:\t",train_a.kernel_)
print("valence:\t",train_v.kernel_)

arousal:	 RationalQuadratic(alpha=0.0304, length_scale=3.4)
valence:	 RationalQuadratic(alpha=0.0304, length_scale=3.4)


AROUSAL

In [32]:
df3_a = df3_a.round(2)

display(df3_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.01,0.16,0.50,-0.53,-0.35,-0.16,-0.84,0.31,-0.33,-0.07,...,0.49,0.11,-0.25,-0.30,-0.11,0.01,0.34,0.44,-0.24,-0.32
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.68


VALENCE

In [33]:
df3_v = df3_v.round(2)

display(df3_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.22,0.22,0.39,-0.36,-0.08,-0.02,-0.36,0.37,0.12,0.12,...,0.48,0.50,-0.28,-0.19,-0.06,0.08,0.21,0.14,-0.32,-0.13
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.38


REGRESSION

In [34]:
# scores_a = regresion(svr_case3_a,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
# scores_v = regresion(svr_case3_v,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [35]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [36]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [37]:
# save_predict(df3_a.round(2),df3_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence

# CASE 4

In [38]:
datafitur =  "case4" #CASE 3
data,arousal_mean,valence_mean = load(datafitur) #Load
print(data.shape)

train_data_a, test_data_a, train_values_a, test_values_a = split(data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #split arousal
train_data_v, test_data_v, train_values_v, test_values_v = split(data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #split valence

(744, 61)


INITIAL MODEL

In [39]:
# svr_case4 = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Arousal

# svr_case1_v = GPR(kernel=kernels, alpha=1e-10, 
#                     optimizer='fmin_l_bfgs_b', 
#                     n_restarts_optimizer=5, 
#                     normalize_y=True, copy_X_train=True, 
#                     random_state=1000) #init Valence

In [40]:
print("arousal:\t",gpr.kernel)
# print("valence:\t",svr_case1_v.kernel)

arousal:	 RationalQuadratic(alpha=1, length_scale=1)


TRAIN

In [41]:
# #Predict Arousal
a_a,train_a,df4_a = predict(train_data_a, train_values_a, test_data_a, test_values_a, gpr)
# #Predict Valence
a_v,train_v,df4_v = predict(train_data_v, train_values_v, test_data_v, test_values_v, gpr)

r2_a = score(a_a,test_values_a) #Score Arousal
r2_v = score(a_v,test_values_v) #Score Valence

In [42]:
print("arousal:\t",train_a.kernel_)
print("valence:\t",train_v.kernel_)

arousal:	 RationalQuadratic(alpha=0.0394, length_scale=6.13)
valence:	 RationalQuadratic(alpha=0.0394, length_scale=6.13)


AROUSAL

In [43]:
df4_a = df4_a.round(2)

display(df4_a.head())
print("R2 Score Arousal:\t%0.2f" % r2_a)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,-0.07,0.16,0.53,-0.60,-0.32,-0.24,-0.71,0.30,-0.28,-0.03,...,0.55,0.13,-0.21,-0.33,-0.08,0.05,0.38,0.48,-0.32,-0.50
1,0.35,0.26,0.41,-0.56,-0.44,-0.41,-0.59,0.24,-0.41,0.35,...,0.59,-0.00,-0.38,0.24,-0.09,-0.03,-0.06,0.38,-0.50,-0.82


R2 Score Arousal:	0.68


VALENCE

In [44]:
df4_v = df4_v.round(2)

display(df4_v.head())
print("R2 Score Valence:\t%0.2f" % r2_v)

,0,1,2,3,4,5,6,7,8,9,...,214,215,216,217,218,219,220,221,222,223
0,0.15,0.29,0.55,-0.17,0.00,-0.02,-0.39,0.34,0.10,0.06,...,0.30,0.11,-0.23,-0.25,-0.11,0.03,0.04,0.17,-0.29,-0.04
1,-0.66,0.48,0.35,-0.66,0.05,-0.51,-0.08,-0.17,-0.23,0.57,...,0.54,-0.02,-0.72,0.23,-0.32,-0.14,0.51,0.51,-0.26,-0.42


R2 Score Valence:	0.38


REGRESSION

In [45]:
# scores_a = regresion(svr_case4_a,data,minmax_scale(arousal_mean.values,feature_range=(-1, 1))) #Arousal Regression
# scores_v = regresion(svr_case4_v,data,minmax_scale(valence_mean.values,feature_range=(-1, 1))) #Arousal Regression

AROUSAL

In [46]:
# print("R2 Score Valence:\t",scores_a)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_a.mean(), scores_a.std() ))

VALENCE

In [47]:
# print("R2 Score Valence:\t",scores_v)
# print("R2 Score Valence:\t%0.2f (+/- %0.2f)" % (scores_v.mean(), scores_v.std() ))

In [48]:
# save_predict(df4_a.round(2),df4_v.round(2),datafitur)
# save_model(train_a,datafitur,'arousal/') #save model Arousal
# save_model(train_v,datafitur,'valence/') #save model Valence